In [7]:
import pandas as pd
import numpy as np

!pip install pytorch_pretrained_bert
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
     
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from nltk.tokenize import word_tokenize 
!pip install krovetzstemmer
import krovetzstemmer
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

!pip install emoji
import emoji

from nltk.corpus import stopwords
from krovetzstemmer import Stemmer
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
!pip install -q keras
import keras
from os import path
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print(accelerator)
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras import backend as K
from keras.models import Sequential
from keras import layers

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 240 kB 32.2 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=57581dc8a6f9ba2a742ea6dd8ac56f32610c4120c50e3f1ecba12b4ca0f9ef1b
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji
cu80
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
olid_training=pd.read_csv("/content/drive/MyDrive/TURKISH-DATA/offenseval-tr-training-v1.tsv",sep="\t")
test=pd.read_csv('/content/drive/MyDrive/TURKISH-DATA/offenseval-tr-testset-v1.tsv',sep="\t")

In [33]:
df_majority = olid_training[olid_training.subtask_a=='NOT']
df_minority = olid_training[olid_training.subtask_a=='OFF']

In [34]:
print(len(df_majority))
print(len(df_minority))

25231
6046


In [35]:
X=olid_training[["id","tweet","subtask_a"]] 
Y=olid_training["subtask_a"]


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)

In [36]:
y_train_encoded = [1 if i == 'OFF' else 0 for i in y_train]
y_test_encoded = [ 1  if i == 'OFF' else 0 for i in y_test]

In [37]:
def preprocess_tweet(tweet):
    # Split tweet into tokens
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(tweet)

    # Remove URLs and mentions
    tokens = [token for token in tokens if not token.startswith('http') and not token.startswith('@')]

    # Decode emojis
    tokens = [emoji.demojize(token) for token in tokens]

    # Remove punctuation and non-alphanumeric characters
    tokens = [token for token in tokens if token.isalnum()]

    # Convert all tokens to lowercase
    tokens = [token.lower() for token in tokens]

    # Remove stop words
    stop_words = set(stopwords.words('turkish'))
    tokens = [token for token in tokens if token not in stop_words]

    tokens = [token for token in tokens if ((token != "url"))]
    return tokens

In [38]:
# 数据清洗

filtered_tweets=[]
for tweet in X_train["tweet"]:
    filtered_tweets.append(preprocess_tweet(tweet))
X_train["tweet_initial"] = filtered_tweets


filtered_tweets=[]
for tweet in X_test["tweet"]:
    filtered_tweets.append(preprocess_tweet(tweet))
X_test["tweet_initial"] = filtered_tweets




#FOR TEST

z=[]
for tweet in X_train["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_train["tweet_initial_nontoken"]=z



#FOR TEST

z=[]
for tweet in X_test["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_test["tweet_initial_nontoken"]=z

In [39]:
X_test

,id,tweet,subtask_a,tweet_initial,tweet_initial_nontoken
30789,38826,@USER ah ya ederiz bayram hediyesi olur:)),NOT,"[ah, ederiz, bayram, hediyesi, olur]",ah ederiz bayram hediyesi olur
24421,27326,@USER @USER Bu din kuruldu adı da lezbiyenlik ...,NOT,"[din, kuruldu, adı, lezbiyenlik, aşırı, iyi, din]",din kuruldu adı lezbiyenlik aşırı iyi din
25180,30778,@USER Dema kırmızı olanındanmi yeşil şu yemekl...,NOT,"[dema, kırmızı, olanındanmi, yeşil, yemekle, y...",dema kırmızı olanındanmi yeşil yemekle yeşille...
17507,23825,"Daha ; kaç hayatın sönmesini, kaç aileni...",NOT,"[kaç, hayatın, sönmesini, kaç, ailenin, umutla...",kaç hayatın sönmesini kaç ailenin umutların kı...
27265,29327,@USER Yahu düdüğü daha küçük assalarmış 😂 düdü...,NOT,"[yahu, düdüğü, küçük, assalarmış, düdük, takan...",yahu düdüğü küçük assalarmış düdük takan kramp...
...,...,...,...,...,...
30494,13383,Zekkoyla istihareye yatacaz ama kim için yapac...,NOT,"[zekkoyla, istihareye, yatacaz, yapacağımızı, ...",zekkoyla istihareye yatacaz yapacağımızı bilmiyoz
2859,31810,bu hayat beni hep karaktersizlerle aynı ortama...,OFF,"[hayat, beni, karaktersizlerle, aynı, ortama, ...",hayat beni karaktersizlerle aynı ortama sokuyor
29977,14514,deminden beri kendi fotoğraflarıma bakıp mutlu...,OFF,"[deminden, beri, kendi, fotoğraflarıma, bakıp,...",deminden beri kendi fotoğraflarıma bakıp mutlu...
6494,16134,taehyung ağzına vura vura sevme isteğimi kabar...,NOT,"[taehyung, ağzına, vura, vura, sevme, isteğimi...",taehyung ağzına vura vura sevme isteğimi kabar...


In [40]:
test

,id,tweet
0,41993,@USER Sayın başkanım bu şekilde devam inşallah👏
1,23000,"Herkes gevşekliği kadar duyar kasıyor,hayat bö..."
2,42478,Olgun ilişkisi olan arkadaş size en güzel hedi...
3,21748,@USER @USER Burada atıp tutacağına o kötü koşu...
4,13607,@USER İşte o onur dediğin sende yok sorun o işte
...,...,...
3510,27484,@USER Çok iyi oyuncu. Serdar’dan çok çok iyi
3511,41742,@USER bu gün mənim Bəbəşimin ad günüdü💜💜💜💜. Tə...
3512,45705,bu gecelik bu kadar yarın gün içerisinde göz...
3513,29225,@USER Okulu bırak o zaman fbshshsb


## BERT-GREEK

In [16]:
# Commented out IPython magic to ensure Python compatibility.
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
!pip install transformers
!pip install sentencepiece
from transformers import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 31.3 MB/s 
     |████████████████████████████████| 182 kB 75.5 MB/s 
     |████████████████████████████████| 7.6 MB 44.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 28.1 MB/s 


In [41]:
#Data pre-processing before BERT
tweets_test = X_test.tweet_initial_nontoken.values


ALL_DATA = pd.concat([X_train,X_test])
ALL_LABEL= y_train_encoded + y_test_encoded

tweets_train = ALL_DATA.tweet_initial_nontoken.values


tweets_train = ["[CLS] " + tweet_initial_nontoken + " [SEP]" for tweet_initial_nontoken in tweets_train]
tweets_test = ["[CLS] " + tweet_initial_nontoken + " [SEP]" for tweet_initial_nontoken in tweets_test]
# X = tweets_train + tweets_test
X = tweets_train
# Y=y_train_encoded + y_test_encoded


# X=tweets_train
Y=np.array(ALL_LABEL)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)

tokenized_X = [tokenizer.tokenize(sent) for sent in X]


print ("Tokenize the first sentence:")
print (tokenized_X[4])

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loading file to

Tokenize the first sentence:
['[CLS]', 'yan', '##ı', '##ma', 'can', '##ı', '##ma', 'so', '##ku', '##l', 'sar', '##k', '##ı', 'soy', '##les', '##in', 'ka', '##h', '##ka', '##han', '[SEP]']


In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")


In [42]:
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_X]

# Pad our input tokens
MAX_LEN=50
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post",value=0)

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [int(i>0) for i in seq]
  attention_masks.append(seq_mask)


# Use train_test_split to split our data into train and validation sets for training
#train_inputs = input_ids[0:13240]
#validation_inputs=input_ds[13240:]
#train_labels=Y[0:13240]
#validation_labels=Y[13240:]
#train_masks=[atten]


#küçük bir hesaplama gerekiyor!! test_size = 860/14100
# train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, Y, 
#                                                             random_state=2018, test_size=0.15)
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, Y, test_size=0.15)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, test_size=0.15)
# train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
#                                              random_state=2018, test_size=0.15)

print(train_inputs.shape)
print(validation_inputs.shape)
print(train_labels)
print(validation_labels)

# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 16

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

from transformers import BertForSequenceClassification, AdamW, BertConfig

(26585, 50)
(4692, 50)
[0 0 0 ... 0 0 1]
[0 0 1 ... 0 0 0]


In [43]:
Bertgreek_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pytorch_model.bin
Some weights of the model check

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [44]:
# Tell pytorch to run this model on the GPU.
Bertgreek_model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [45]:

# Get all of the model's parameters as a list of tuples.
params = list(Bertgreek_model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [48]:

# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(Bertgreek_model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


In [49]:

import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

"""### Training"""

import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    Bertgreek_model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        Bertgreek_model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = Bertgreek_model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask,labels=b_labels
                    )
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss = total_loss + loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()
        #loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(Bertgreek_model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    Bertgreek_model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = Bertgreek_model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")




======== Epoch 1 / 4 ========
Training...
  Batch    40  of  1,662.    Elapsed: 0:00:07.
  Batch    80  of  1,662.    Elapsed: 0:00:13.
  Batch   120  of  1,662.    Elapsed: 0:00:20.
  Batch   160  of  1,662.    Elapsed: 0:00:26.
  Batch   200  of  1,662.    Elapsed: 0:00:33.
  Batch   240  of  1,662.    Elapsed: 0:00:39.
  Batch   280  of  1,662.    Elapsed: 0:00:46.
  Batch   320  of  1,662.    Elapsed: 0:00:52.
  Batch   360  of  1,662.    Elapsed: 0:00:59.
  Batch   400  of  1,662.    Elapsed: 0:01:05.
  Batch   440  of  1,662.    Elapsed: 0:01:12.
  Batch   480  of  1,662.    Elapsed: 0:01:18.
  Batch   520  of  1,662.    Elapsed: 0:01:25.
  Batch   560  of  1,662.    Elapsed: 0:01:31.
  Batch   600  of  1,662.    Elapsed: 0:01:38.
  Batch   640  of  1,662.    Elapsed: 0:01:44.
  Batch   680  of  1,662.    Elapsed: 0:01:51.
  Batch   720  of  1,662.    Elapsed: 0:01:57.
  Batch   760  of  1,662.    Elapsed: 0:02:04.
  Batch   800  of  1,662.    Elapsed: 0:02:10.
  Batch   840  of

In [50]:
PATH='/content/cache/BERT-TUR-4-EPOCH-FULL'

torch.save(Bertgreek_model, PATH)

In [56]:
# Model class must be defined somewhere
PATH='/content/cache/BERT-TUR-4-EPOCH-FULL'

greekbert_model_loaded = torch.load(PATH)
# greekbert_model_loaded.eval()

In [52]:
import csv
# 正确的label
# labels = np.ones(len(test))
# labels
Y_TEST=pd.read_csv('/content/drive/MyDrive/TURKISH-DATA/offenseval-tr-labela-v1.tsv',sep=',',encoding='utf8',quoting=csv.QUOTE_NONE,header=None)
labels = [1 if i ==  'OFF' else 0 for i in Y_TEST[1]]

truetrue_labels = np.array(labels)

In [70]:
test=pd.read_csv('/content/drive/MyDrive/TURKISH-DATA/offenseval-tr-testset.tsv',sep="\t")
true_label = test["label"]
labels = [ 1  if i == 'OFF' else 0 for i in true_label]
truetrue_labels = np.array(labels)

In [72]:
# 数据清洗

filtered_tweets=[]
for tweet in test["tweet"]:
    filtered_tweets.append(preprocess_tweet(tweet))
test["tweet_initial"] = filtered_tweets

#FOR TEST

z=[]
for tweet in test["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
test["tweet_initial_nontoken"]=z

In [73]:
test

,id,tweet,label,tweet_initial,tweet_initial_nontoken
0,41993,@USER Sayın başkanım bu şekilde devam inşallah👏,NOT,"[sayın, başkanım, şekilde, devam, inşallah]",sayın başkanım şekilde devam inşallah
1,23000,"Herkes gevşekliği kadar duyar kasıyor,hayat bö...",NOT,"[herkes, gevşekliği, kadar, duyar, kasıyor, ha...",herkes gevşekliği kadar duyar kasıyor hayat böyle
2,42478,Olgun ilişkisi olan arkadaş size en güzel hedi...,NOT,"[olgun, ilişkisi, olan, arkadaş, size, güzel, ...",olgun ilişkisi olan arkadaş size güzel hediye
3,21748,@USER @USER Burada atıp tutacağına o kötü koşu...,OFF,"[burada, atıp, tutacağına, kötü, koşullarda, 3...",burada atıp tutacağına kötü koşullarda 3 kuruş...
4,13607,@USER İşte o onur dediğin sende yok sorun o işte,OFF,"[i̇şte, onur, dediğin, sende, yok, sorun, işte]",i̇şte onur dediğin sende yok sorun işte
...,...,...,...,...,...
3510,27484,@USER Çok iyi oyuncu. Serdar’dan çok çok iyi,NOT,"[iyi, oyuncu, serdar, dan, iyi]",iyi oyuncu serdar dan iyi
3511,41742,@USER bu gün mənim Bəbəşimin ad günüdü💜💜💜💜. Tə...,NOT,"[gün, mənim, bəbəşimin, ad, günüdü, təbrik, ed...",gün mənim bəbəşimin ad günüdü təbrik edirəm gö...
3512,45705,bu gecelik bu kadar yarın gün içerisinde göz...,NOT,"[gecelik, kadar, yarın, gün, içerisinde, göze,...",gecelik kadar yarın gün içerisinde göze çarpan...
3513,29225,@USER Okulu bırak o zaman fbshshsb,NOT,"[okulu, bırak, zaman, fbshshsb]",okulu bırak zaman fbshshsb


In [74]:
"""## Testing"""

#Data pre-processing before BERT
tweets_test = test.tweet_initial_nontoken.values


sentences = ["[CLS] " + tweet_initial_nontoken + " [SEP]" for tweet_initial_nontoken in tweets_test]
# labels = np.ones(len(test))

# len(labels)

# tokenize test data
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
MAX_LEN = 50
# Pad our input tokens
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

# create test tensors
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
batch_size = 32  
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)

prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)


# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
greekbert_model_loaded.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = greekbert_model_loaded(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

# Import and evaluate each test batch using Matthew's correlation coefficient
from sklearn.metrics import matthews_corrcoef, accuracy_score
matthews_set = []
for i in range(len(true_labels)):
  matthews = matthews_corrcoef(true_labels[i],
                 np.argmax(predictions[i], axis=1).flatten())
  matthews_set.append(matthews)
  
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]
print(flat_predictions)

# print('Classification accuracy using BERT Fine Tuning: {0:0.2%}'.format(matthews_corrcoef(flat_true_labels, flat_predictions)))
print('Classification accuracy using BERT Fine Tuning: {0:0.2%}'.format(accuracy_score(truetrue_labels, flat_predictions)))
print(classification_report(truetrue_labels,flat_predictions))

prediction_bert_turk=flat_predictions

Predicting labels for 3,515 test sentences...
    DONE.
[0 0 0 ... 0 0 0]
Classification accuracy using BERT Fine Tuning: 79.77%
              precision    recall  f1-score   support

           0       0.80      1.00      0.89      2804
           1       0.00      0.00      0.00       711

    accuracy                           0.80      3515
   macro avg       0.40      0.50      0.44      3515
weighted avg       0.64      0.80      0.71      3515



In [ ]:
# DONE !!!